# Analyse reindexation from ABES - April 2023, corrected in June 2023

# Set project

### Packages

In [26]:
# Import librairies
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings



from utils_metrics import *
from utils_visualization import *

In [27]:
# Suppression des FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

### Graphical parameters

In [28]:
# Parametres graphiques
%matplotlib inline
rcParams = {
    'font.size': 14,
    'font.family':'sans-serif',
    'font.sans-serif':['Liberation Sans'],
    'axes.labelsize': 14,
    'legend.fontsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'figure.max_open_warning': 30}

sns.set(rc=rcParams)
sns.set_style(
    "whitegrid", {
        'axes.edgecolor': 'k',
        'axes.linewidth': 1,
        'axes.grid': True,
        'xtick.major.width': 1,
        'ytick.major.width': 1
        })

sns.set_context(
    "notebook",
    font_scale=1.1,
    rc={"lines.linewidth": 1.5})

pd.set_option('display.max_columns', None)

### Paths

In [29]:
# Set paths
path = "."
os.chdir(path)
data_path = path + "/data"
data_path_reindex = "/home/aurelie/ABES/labo-indexation-ai/data/reindexation/correction_juin2023"
data_path_embed = "/home/aurelie/ABES/labo-indexation-ai/data/embeddings"
output_path = path + "/outputs"
fig_path = path + "/figs"

In [30]:
# Include embeddings
include_embedding = True

# Import data

In [31]:
# Original dataset
reindex = "fusion_fichiers_reindexation.xlsx"
rameau_file = "./extraction/liste_concepts_rameau_sans_vedetteConstruites.csv"
embedding = "embeddings_results.csv"

### Check reindexation files

In [32]:
# Merge with already merged file for concept correction
df = pd.read_excel(os.path.join(data_path_reindex, reindex), index_col=0)
print(df.shape)
df.head()

(112, 16)


,N°ordre,PPN,TITRE,RESUME,INDEX_UNI_AFE,INDEX_AFE,INDEX_UNI_MCR,INDEX_MCR,INDEX_UNI_JMF,INDEX_JMF,INDEX_UNI_LPL,INDEX_LPL,INDEX_UNI_LJZ,INDEX_LJZ,INDEX_UNI_MPD,INDEX_MPD
0,79.0,000308838,Les sommets de l'État : essai sur l'élite du p...,"Du XIXe siècle à nos jours, l'État ""fort"" à la...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Pou...,606 $3027365581$aPouvoir (sciences sociales)$3...,Pouvoir (sciences sociales) -- Classes dirigea...,NaN,NaN,606$3027322610$aHauts-fonctionnairesss$3027226...,Hauts-fonctionnairesss -- France;;Classes diri...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France;;Hauts-fonctionn...,NaN,NaN
1,58.0,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France -- 1958-.... (5e Répub...,606 $3027235149$aIntellectuels$3027226794$yFra...,Intellectuels -- France -- 1945,NaN,NaN,606$027235149$aIntellectuels$3027284948$yFranc...,Intellectuels -- France -- 1958 (5e République...,606$3027235149$aIntellectuels$3027226794$yFran...,Intellectuels -- France;;Vie intellectuelle --...,NaN,NaN
2,59.0,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme,606 $3027321835$aBouddhisme,Bouddhisme,NaN,NaN,606$3027321835$aBouddhisme$3027789853$xEtude e...,Bouddhisme -- Étude et enseignement;;Philosoph...,606$3027321835$aBouddhisme$3030974208$xCoutume...,Bouddhisme -- Coutumes et pratiques;;Bouddhism...,NaN,NaN
3,65.0,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",606$302736576X$aPhilosophie et sciences,Philosophie et sciences,606 $3027248283$aPhilosophie des sciences,Philosophie des sciences,NaN,NaN,606$3027248283$aPhilosophie des sciences\n606$...,Philosophie des sciences;;Futur (philosophie),606$3027582957$aLoi (philosophie)\n606$3027365...,Loi (philosophie);;Philosophie et sciences,NaN,NaN
4,80.0,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,606 $3027234312$aImpérialisme$3027365581$xPouv...,Impérialisme -- Pouvoir (sciences sociales) --...,NaN,NaN,606$3027365581$aPouvoir (sciences sociales)$30...,Pouvoir (sciences sociales) -- Histoire -- Rom...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,NaN,NaN


### Reorganise data

In [33]:
def import_indexation(file):
    df = pd.read_excel(
        os.path.join(data_path_reindex, file), 
        sheet_name="Notices à indexer", 
        engine="openpyxl", 
        converters={"PPN": lambda x: str(x)})
    df.drop(df[df["PPN"].isna()].index, inplace=True) # delete first line corresponding to example
    df["PPN"] = df["PPN"].astype(str)
    
    return df

In [34]:
# Set list of indexeurs
suffix_list = ["AFE", "MCR", "JMF", "LPL", "LJZ", "MPD"]

In [35]:
# Raw initial import 
index_file = '100 NOTICES A INDEXER_AFE.xlsx'
data = import_indexation(index_file)
print(data.shape)
data

(100, 6)


,N°ordre,PPN,TITRE,RESUME,INDEXATION UNIMARC,INDEXATION AFFICHAGE PUBLIC
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire\nPou...
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France‎ -- 1958-.... (5e Répu...
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme
3,65,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",606$302736576X$aPhilosophie et sciences,Philosophie et sciences
4,80,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie)\nEmpereurs -- ...
...,...,...,...,...,...,...
95,38,23690454X,Droit public et patrimoine : le rôle du Consei...,"la 4e de couv. indique : ""L'histoire du droit ...",606$3050573004$aDroit public$3027226794$yFranc...,Droit public --France -- Histoire\nEtat -- Res...
96,95,24155859X,Lars Fredrikson : [exposition Musée d'art mode...,Première grande monographie consacrée à l'œuvr...,606$3027280721$aArtistes$3027622444$ySuède$308...,Artistes -- Suède -- 20e siècle
97,52,25561280X,L'ère des soulèvements : émeutes et confinemen...,"Dès les années 1980, Michel Maffesoli se fait ...",606$3027267547$aRévoltes$3027226794$yFrance$30...,Révoltes -- France -- 2000-….\nRésistance poli...
98,99,26117309X,Monde enchanté : chansons et imaginaires géogr...,Le monde est enchanté par certaines chansons. ...,606$3183790669$aMusique et géographie\n606$302...,Musique et géographie\nRécits de voyages -- Da...


In [37]:
# Remove useless columns
data.drop(columns=["INDEXATION UNIMARC", "INDEXATION AFFICHAGE PUBLIC"], inplace=True)
data

,N°ordre,PPN,TITRE,RESUME
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ..."
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ..."
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon..."
3,65,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod..."
4,80,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...
...,...,...,...,...
95,38,23690454X,Droit public et patrimoine : le rôle du Consei...,"la 4e de couv. indique : ""L'histoire du droit ..."
96,95,24155859X,Lars Fredrikson : [exposition Musée d'art mode...,Première grande monographie consacrée à l'œuvr...
97,52,25561280X,L'ère des soulèvements : émeutes et confinemen...,"Dès les années 1980, Michel Maffesoli se fait ..."
98,99,26117309X,Monde enchanté : chansons et imaginaires géogr...,Le monde est enchanté par certaines chansons. ...


In [38]:
# Recreate indexation files
for suf in suffix_list:
    print(f"Merging {suf} reindexation")
    unimarc = str("INDEX_UNI_" + suf)
    public = str("INDEX_" + suf)
    temp = df[["PPN", unimarc, public]].dropna()
    data = pd.merge(data, temp, on="PPN", how="outer")
    print("dimension after merge: ", data.shape)

Merging AFE reindexation
dimension after merge:  (100, 6)
Merging MCR reindexation
dimension after merge:  (100, 8)
Merging JMF reindexation
dimension after merge:  (100, 10)
Merging LPL reindexation
dimension after merge:  (100, 12)
Merging LJZ reindexation
dimension after merge:  (100, 14)
Merging MPD reindexation
dimension after merge:  (100, 16)


In [39]:
data

,N°ordre,PPN,TITRE,RESUME,INDEX_UNI_AFE,INDEX_AFE,INDEX_UNI_MCR,INDEX_MCR,INDEX_UNI_JMF,INDEX_JMF,INDEX_UNI_LPL,INDEX_LPL,INDEX_UNI_LJZ,INDEX_LJZ,INDEX_UNI_MPD,INDEX_MPD
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Pou...,606 $3027365581$aPouvoir (sciences sociales)$3...,Pouvoir (sciences sociales) -- Classes dirigea...,606$3027223345$aClasses dirigeantes$302779430X...,Classes dirigeantes -- Relations avec l'État -...,606$3027322610$aHauts-fonctionnairesss$3027226...,Hauts-fonctionnairesss -- France;;Classes diri...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France;;Hauts-fonctionn...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Éli...
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France -- 1958-.... (5e Répub...,606 $3027235149$aIntellectuels$3027226794$yFra...,Intellectuels -- France -- 1945,606$3028696980$aIntellectuels français$3049647...,Intellectuels français -- Sociologie ;;Intelle...,606$027235149$aIntellectuels$3027284948$yFranc...,Intellectuels -- France -- 1958 (5e République...,606$3027235149$aIntellectuels$3027226794$yFran...,Intellectuels -- France;;Vie intellectuelle --...,606 ##‎$3027235149Intellectuels‎$3027284948$yF...,Intellectuels -- France -- 1958-.... (5e Répub...
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme,606 $3027321835$aBouddhisme,Bouddhisme,606$3027431215$aPhilosophie bouddhique \n606$3...,Philosophie bouddhique ;;Spiritualité bouddhique,606$3027321835$aBouddhisme$3027789853$xEtude e...,Bouddhisme -- Étude et enseignement;;Philosoph...,606$3027321835$aBouddhisme$3030974208$xCoutume...,Bouddhisme -- Coutumes et pratiques;;Bouddhism...,606$3027321835$aBouddhisme,Bouddhisme
3,65,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",606$302736576X$aPhilosophie et sciences,Philosophie et sciences,606 $3027248283$aPhilosophie des sciences,Philosophie des sciences,606$3027286584$aCréativité$3027248275$xScience...,Créativité -- Sciences ;;Devenir (philosophie)...,606$3027248283$aPhilosophie des sciences\n606$...,Philosophie des sciences;;Futur (philosophie),606$3027582957$aLoi (philosophie)\n606$3027365...,Loi (philosophie);;Philosophie et sciences,606$302736576X$aPhilosophie et sciences\n606‎$...,Philosophie et sciences;;Temps;;Incertitude
4,80,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,606 $3027234312$aImpérialisme$3027365581$xPouv...,Impérialisme -- Pouvoir (sciences sociales) --...,606$3027873757$aCulte impérial$3027243001$yRom...,Culte impérial -- Rome ;;Politique et gouverne...,606$3027365581$aPouvoir (sciences sociales)$30...,Pouvoir (sciences sociales) -- Histoire -- Rom...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,606$3154298611$aPersonnalisation du pouvoir‎$3...,Personnalisation du pouvoir -- Rome -- Histoir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,38,23690454X,Droit public et patrimoine : le rôle du Consei...,"la 4e de couv. indique : ""L'histoire du droit ...",606$3050573004$aDroit public$3027226794$yFranc...,Droit public --France -- Histoire;;État -- Res...,606 $3027797856$aPatrimoine culturel$305057300...,Patrimoine culturel -- Droit public;;Patrimoin...,606$3027797856$aPatrimoine culturel$3050573004...,Patrimoine culturel -- Droit public ;;Conseill...,606$3027797856$aPatrimoine culturel$3027466868...,Patrimoine culturel -- Droit -- France;;Patrim.

In [41]:
#Save final reindexation file
data.to_csv(os.path.join(data_path, "reindexation_final_juin2023.csv"))

## Check files

In [42]:
# Import rameau list
ram = pd.read_csv(os.path.join(data_path, rameau_file), encoding="latin-1")
print("Nombre de notices différentes: ", ram.shape[0])
ram.head(10)

Nombre de notices différentes:  85876


,PPN,NOM
0,157992527,Kirp?n
1,110140494,Militaires artistes
2,028492161,Militaires romains
3,028521757,Militaires prussiens
4,029895561,Sa-skya-pa
5,031875459,Militaires réunionnais
6,032370083,Construction à l'épreuve de la sécheresse
7,032878117,Missionnaires suisses
8,034423982,Militaires ivoiriens
9,034686940,Outils à métaux


In [43]:
# Embeddings
emb = pd.read_csv(os.path.join(data_path_embed, embedding), index_col=0)
emb["PPN"] = emb["PPN"].astype(str)
emb["embedding_chains"] = emb["embedding_chains"].apply(lambda x: eval(x))
emb["embeddings_autorites"] = emb["embeddings_autorites"].apply(lambda x: eval(x))
print("Dimension of the dataset: ", emb.shape)
emb.head()

Dimension of the dataset:  (100, 3)


,PPN,embedding_chains,embeddings_autorites
0,000308838,"[État, Administration publique, Fédéralisme, F...","[État, Administration publique, Fédéralisme, F..."
1,00094758X,"[Dollar américain, Système monétaire internati...","[Dollar américain, Système monétaire internati..."
2,003632806,"[Intellectuels -- Activité politique, Intellec...","[Intellectuels, Activité politique, Intellectu..."
3,047450037,"[Bouddhisme -- Anthologies, Bouddhisme -- Livr...","[Bouddhisme, Anthologies, Bouddhisme, Livres s..."
4,05224170X,"[Apprentissage par problèmes, Psychologie de l...","[Apprentissage par problèmes, Psychologie de l..."


In [45]:
if include_embedding:
    data = data.merge(emb, on="PPN", how="inner")
    print("Dimension after merge with embeddings: ", data.shape)
    # Check merging
    data.head(5)
    # Save merged file
    data.to_csv(os.path.join(data_path, "data_with_reindexation_and_embeddings.csv"))

Dimension after merge with embeddings:  (112, 16)


,N°ordre,PPN,TITRE,RESUME,INDEX_UNI_AFE,INDEX_AFE,INDEX_UNI_MCR,INDEX_MCR,INDEX_UNI_JMF,INDEX_JMF,INDEX_UNI_LPL,INDEX_LPL,INDEX_UNI_LJZ,INDEX_LJZ,INDEX_UNI_MPD,INDEX_MPD,embedding_chains,embeddings_autorites
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Pou...,606 $3027365581$aPouvoir (sciences sociales)$3...,Pouvoir (sciences sociales) -- Classes dirigea...,606$3027223345$aClasses dirigeantes$302779430X...,Classes dirigeantes -- Relations avec l'État -...,606$3027322610$aHauts-fonctionnairesss$3027226...,Hauts-fonctionnairesss -- France;;Classes diri...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France;;Hauts-fonctionn...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Éli...,"[État, Administration publique, Fédéralisme, F...","[État, Administration publique, Fédéralisme, F..."
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France -- 1958-.... (5e Répub...,606 $3027235149$aIntellectuels$3027226794$yFra...,Intellectuels -- France -- 1945,606$3028696980$aIntellectuels français$3049647...,Intellectuels français -- Sociologie ;;Intelle...,606$027235149$aIntellectuels$3027284948$yFranc...,Intellectuels -- France -- 1958 (5e République...,606$3027235149$aIntellectuels$3027226794$yFran...,Intellectuels -- France;;Vie intellectuelle --...,606 ##‎$3027235149Intellectuels‎$3027284948$yF...,Intellectuels -- France -- 1958-.... (5e Répub...,"[Intellectuels -- Activité politique, Intellec...","[Intellectuels, Activité politique, Intellectu..."
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme,606 $3027321835$aBouddhisme,Bouddhisme,606$3027431215$aPhilosophie bouddhique \n606$3...,Philosophie bouddhique ;;Spiritualité bouddhique,606$3027321835$aBouddhisme$3027789853$xEtude e...,Bouddhisme -- Étude et enseignement;;Philosoph...,606$3027321835$aBouddhisme$3030974208$xCoutume...,Bouddhisme -- Coutumes et pratiques;;Bouddhism...,606$3027321835$aBouddhisme,Bouddhisme,"[Bouddhisme -- Anthologies, Bouddhisme -- Livr...","[Bouddhisme, Anthologies, Bouddhisme, Livres s..."
3,65,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",606$302736576X$aPhilosophie et sciences,Philosophie et sciences,606 $3027248283$aPhilosophie des sciences,Philosophie des sciences,606$3027286584$aCréativité$3027248275$xScience...,Créativité -- Sciences ;;Devenir (philosophie)...,606$3027248283$aPhilosophie des sciences\n606$...,Philosophie des sciences;;Futur (philosophie),606$3027582957$aLoi (philosophie)\n606$3027365...,Loi (philosophie);;Philosophie et sciences,606$302736576X$aPhilosophie et sciences\n606‎$...,Philosophie et sciences;;Temps;;Incertitude,"[Philosophie des sciences, Philosophie et scie...","[Philosophie des sciences, Philosophie et scie..."
4,80,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,606 $3027234312$aImpérialisme$3027365581$xPouv...,Impérialisme -- Pouvoir (sciences sociales) --...,606$3027873757$aCulte impérial$3027243001$yRom...,Culte impérial -- Rome ;;Politique et gouverne...,606$3027365581$aPouvoir (sciences sociales)$30...,Pouvoir (sciences sociales) -- Histoire -- Rom...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,606$3154298611$aPersonnalisation du pouvoir‎$3...,Personnalisation du pouvoir -- Rome -- Histoir...,"[Voies romaines, Colonies romaines, Romanisati...","[Voies romaines, Colonies romaines, Romanisati..."
